In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# ALGORITMO DE CARGA DE PROBABILIDAD

In [ ]:
%load_ext qat.core.magic

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os

In [ ]:
sys.path.append('/home/gferro/Documentos/NEASQC/PhaseAmplification/')
sys.path.append('/home/cesga/gferro/NEASQC/PhaseAmplification/')

In [ ]:
#QPU connection
try:
    from qat.qlmaas import QLMaaSConnection
    connection = QLMaaSConnection()
    LinAlg = connection.get_qpu("qat.qpus:LinAlg")
    lineal_qpu = LinAlg()
    print('QLM Used')
except (ImportError, OSError) as e:
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()
    print('PyLinalg Used')

In [ ]:
from AuxiliarFunctions import  get_histogram, PostProcessResults, TestBins, LeftConditionalProbability, RunJob
def p(x):
    return x*x
def f(x):
    return np.sin(x)

## Rutina de Rotación Controlada

A mi entender la pieza clave del circuito es la que permite aplicar una rotación controlada sobre un qbit target utilizando un estado de control de n qbits. Básicamente lo que necesito es dado uno de los posibles ángulos de rotación:

$$\theta^{m}_{i} = \arccos{\sqrt{f^{m}(i)}} \,\, donde \,i=0,1,...2^{m-1}-1$$

necesito poder aplicar una rotación sobre el qbit target cuando los n qbits anteriores estén en el estado $|i\rangle$ del siguiente modo:

$$|i\rangle\hat{R}_{y}(2\theta^{m}_{i})|0\rangle_{target}$$

La siguiente rutina hace eso: recibe un número de qbits en el que se debe codificar el estado de control y un ángulo de rotación. La función prepara **nqbits** para que cuando el estado de esos qbits sea el de *control* entonces se aplique la rotación deseada. 

Para ello el *estado de control* tiene que ser un entero y la función lo descompone en un número binario de **nqbits** dígitos. Con un bucle sobre estos dígitos se crea un circuito que generará **nqbits** igual a 1 cuando en esos qbits entre el *estado de control*. 

Con esta configuración podemos crear una rotación alrededor del eje Y del ángulo deseado, controlada por los **nqbits**, sobre un qbit target adicional. 

Finalmente debemos deshacer la preparación del *estado de control*.

## Rutina  a puerta

In [ ]:
from qat.lang.AQASM import AbstractGate

In [ ]:
def CRBS_generatorINT(nqbits, ControlState, theta):
    """ 
    This functions condify a input ControlState using N qbits and
    apply a controlled Rotation of an input angle theta by the ControlState
    on one aditional qbit.
    Inputs:
    * nqbits: int. Number of qbits needed for codify the ControlState. 
    * ControlState: int. State for controlling the of the controlled Rotation.
    * theta: float. Rotation angle    
    """
    from qat.lang.AQASM import QRoutine, RY
    qrout = QRoutine()
    from qat.lang.AQASM.qint import QInt
    qcontrol = qrout.new_wires(nqbits, QInt)#, reverse_bit_order=True)
    qtarget = qrout.new_wires(1)
    #c_i_RY = RY(theta).ctrl()
    expresion = (qcontrol==ControlState)
    with qrout.compute():
        qbit4cr=expresion.evaluate()
    qrout.apply(RY(theta).ctrl(), qbit4cr, qtarget)
    qrout.uncompute()
    qrout.free_ancillae(qbit4cr)
    return qrout

from qat.lang.AQASM import AbstractGate
#Using generator function an abstract gate is created
CRBS_gateINT = AbstractGate(
    "CRBS", 
    [int, int, float], 
    circuit_generator = CRBS_generatorINT,
    arity = lambda x, y, z: x+1
)

In [ ]:
def LoadP_Gate(ProbabilityArray):
    """
    Creates a customized AbstractGate for loading a discretized Probability
    Inputs:
        * ProbabilityArray: numpy array. Numpy array with the discretized probability to load. The number of qbits will be log2(len(ProbabilityArray)). 
    Output:
        * AbstractGate: AbstractGate customized 
    """
    def P_generator():
        """
        Function generator for the AbstractGate that allows the loading of a discretized Probability in a Quantum State.
        Output:
            * qrout: Quantum Routine
        """
    
        #ProbabilityArray = Dictionary['array']
        n_qbits = TestBins(ProbabilityArray, 'Probability')
        from qat.lang.AQASM import QRoutine, RY
        qrout = QRoutine()
        qbits = qrout.new_wires(n_qbits)
        nbins = len(ProbabilityArray)
    
        for i in range(0, n_qbits):
            ConditionalProbability = LeftConditionalProbability(i, ProbabilityArray)
            Thetas = 2.0*(np.arccos(np.sqrt(ConditionalProbability)))
    
            if i == 0:
                #The first qbit is a typical y Rotation
                qrout.apply(RY(Thetas[0]), qbits[0])
            else:
                #The different rotations should be applied  over the i+1 qbit.
                #Each rotation is controlled by all the posible states formed with i qbits
                for j, theta in enumerate(Thetas):
                    #Next lines do the following operation: |j> x Ry(2*\theta_{j})|0>
                    gate = CRBS_gateINT(i, j, theta)
                    qrout.apply(gate, qbits[:i+1])
        return qrout
    P_Gate = AbstractGate(
        "P_Gate",
        [],
        circuit_generator = P_generator,
        arity = TestBins(ProbabilityArray, 'Probability')
    )
    return P_Gate()


In [ ]:
#number of Qbits for the circuit
n_qbits = 4
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

In [ ]:
P_Gate = LoadP_Gate(p_X)

In [ ]:
%qatdisplay P_Gate --depth 2

In [ ]:
from qat.lang.AQASM import X, RY, Program

qprog = Program()
qbits = qprog.qalloc(P_Gate.arity)
qprog.apply(P_Gate, qbits)
#Create the circuit from the program
circuit = qprog.to_circ()

#Display the circuit
#%qatdisplay circuit
%qatdisplay circuit --depth 0
job = circuit.to_job()
result = lineal_qpu.submit(job)
PGATEINT_results = PostProcessResults(RunJob(result))

In [ ]:
circuit.statistics()

In [ ]:
np.isclose(PGATEINT_results['Probability'], p_X).all()

In [ ]:
from qat.pbo import GraphCircuit
from qat.lang.AQASM import Program, H, X

#
# Write circuit
#

# Define initial circuit (X - H - H circuit)
prog = Program()
qubit = prog.qalloc(1)
prog.apply(X, qubit)
prog.apply(H, qubit)
prog.apply(H, qubit)
circ = prog.to_circ()

#
# Optimize circuit
#

# Create a graph object and load circuit
graph = GraphCircuit()
graph.load_circuit(circ)

# Define two patterns
left_pattern = [("H", [0]), ("H", [0])]
right_pattern = []

# Replace left_pattern by right_pattern, i.e. the old one by the new one
graph.replace_pattern(left_pattern, right_pattern)